In [1]:
import sys
sys.path.insert(1, "../..")
import numpy as np
import gillespy2
from gillespy2 import (
    NumPySSASolver,
    ODESolver,
    TauLeapingSolver,
    TauHybridSolver,
    SSACSolver,
    ODECSolver,
    TauLeapingCSolver,
    TauHybridCSolver
)

In [2]:
def build_from_solver_results(name, result, rc, model):
        """
        Build a gillespy2.Results object using the provided solver results.
        :param solver: The solver used to run the simulation.
        :type solver: gillespy2.GillesPySolver
        """
        if rc == 33:
            from gillespy2.core import log
            log.warning('GillesPy2 simulation exceeded timeout.')
        if hasattr(result[0], 'shape'):
            return result
        if len(result) > 0:
            results_list = []
            for i in range(0, len(result)):
                temp = gillespy2.Trajectory(data=result[i], model=model, solver_name=name, rc=rc)
                results_list.append(temp)

            return gillespy2.Results(results_list)
        else:
            raise ValueError("number_of_trajectories must be non-negative and non-zero")

# Models

## Degradation Model

In [3]:
class Degradation(gillespy2.Model):
    def __init__(self, model_name="Degradation"):
        super().__init__(name=model_name)
        
        A = gillespy2.Species(name="A", initial_value=2000)
        self.add_species(A)
        
        k1 = gillespy2.Parameter(name="k1", expression="0.1")
        self.add_parameter(k1)
        
        r1 = gillespy2.Reaction(name="r1", rate="k1", reactants={"A":1})
        self.add_reaction(r1)

In [4]:
d_model = Degradation()

## Boundary Condition Test Model

In [ ]:
class BoundaryConditionTestModel(gillespy2.Model):
    def __init__(self):
        gillespy2.Model.__init__(self, name="BoundaryConditionTestModel")
        s1 = gillespy2.Species(name="S1", boundary_condition=True, initial_value=0.001, mode='continuous')
        s2 = gillespy2.Species(name="S2", boundary_condition=False, initial_value=0.002, mode='continuous')
        s3 = gillespy2.Species(name="S3", boundary_condition=False, initial_value=0.001, mode='continuous')
        s4 = gillespy2.Species(name="S4", boundary_condition=True, initial_value=0.002, mode='continuous')
        self.add_species([s1, s2, s3, s4])

        k1 = gillespy2.Parameter(name="k1", expression="0.75")
        k2 = gillespy2.Parameter(name="k2", expression="0.25")
        self.add_parameter([k1, k2])

        reaction1 = gillespy2.Reaction(name="reaction1", propensity_function="vol*k1*S1*S2",
                                       reactants={s1: 1, s2: 1},
                                       products={s3: 1})
        reaction2 = gillespy2.Reaction(name="reaction2", propensity_function="vol*k2*S3",
                                       reactants={s3: 1},
                                       products={s1: 1, s2: 1})
        reaction3 = gillespy2.Reaction(name="reaction3", propensity_function="vol*k2*S3",
                                       reactants={s1: 1, s2: 2, s3: 1},
                                       products={s4: 1})
        self.add_reaction([reaction1, reaction2, reaction3])

        self.timespan(np.linspace(0, 20, 51))

In [ ]:
bct_model = BoundaryConditionTestModel()

## Rate Rule Test Model

In [ ]:
class RateRuleTestModel(gillespy2.Model):
    def __init__(self):
        gillespy2.Model.__init__(self, name="RateRuleTestModel")
        s1 = gillespy2.Species(name="S1", initial_value=0.015, mode='continuous')
        s2 = gillespy2.Species(name="S2", initial_value=0.0, mode='continuous')
        s3 = gillespy2.Species(name="S3", initial_value=1.0, mode='continuous')
        self.add_species([s1, s2, s3])

        k1 = gillespy2.Parameter(name="k1", expression="1.0")
        self.add_parameter([k1])

        rule1 = gillespy2.RateRule(name="rule1", variable="S1", formula="-k1*S1")
        rule2 = gillespy2.RateRule(name="rule2", variable="S2", formula="k1*S1")
        rule3 = gillespy2.RateRule(name="rule3", variable="S3", formula="0.015-(S1+S2)")
        self.add_rate_rule([rule1, rule2, rule3])

        self.timespan(np.linspace(0, 20, 51))

In [ ]:
rrt_model = RateRuleTestModel()

# Testing Increment Fix

### Test `model.run()` with increment passed and default tspan

In [ ]:
results = d_model.run()
results.plot()

### Test `model.run(solver=solver)` with increment passed and default tspan

In [ ]:
from gillespy2 import NumPySSASolver
results = d_model.run(solver=NumPySSASolver)
results.plot()

### Test `solver.run(model=model)` with increment passed and default tspan

In [ ]:
solvers = [NumPySSASolver, ODESolver, TauLeapingSolver, TauHybridSolver,
    SSACSolver, ODECSolver, TauLeapingCSolver, TauHybridCSolver]

for solver in solvers:
    try:
        sol_results, rc = solver.run(model=d_model, t=100, increment=1)
        results = build_from_solver_results(d_model, sol_results, rc, solver.name)
        results.plotplotly(title=solver.name)
    except Exception as err:
        print(solver.name)
        print(err)

Exception in thread Thread-9:
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.9/3.9.2_4/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 954, in _bootstrap_inner
    self.run()
  File "/usr/local/Cellar/python@3.9/3.9.2_4/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "../../gillespy2/solvers/cpp/c_decoder.py", line 237, in read
    self.callback(self.trajectories[traj_id][t][0],
  File "../../gillespy2/solvers/cpp/ssa_c_solver.py", line 86, in <lambda>
    decoder.with_callback(lambda curr_time, curr_stat, trajectory_base=decoder.trajectories: print(curr_time, curr_state, trajectory_base))
NameError: name 'curr_state' is not defined


# Test Hybrid Rate Rules

In [ ]:
from gillespy2 import TauHybridCSolver

In [ ]:
solver = TauHybridCSolver(model=bct_model)
results = bct_model.run(solver=solver)
results.plot()

In [ ]:
solver = TauHybridCSolver(model=rrt_model)
results = rrt_model.run(solver=solver)
results.plot()

# Testing Output Adaptors

In [ ]:
from gillespy2 import SSACSolver

In [ ]:
d_model.timespan(np.arange(0, 101))
solver = SSACSolver(model=d_model)
results = d_model.run(solver=solver)

In [ ]:
print(results.data)
print(len(results.data[0]["A"]))
print(results.data[0]["time"])
print(results.data[0]["A"])

In [ ]:
results.plotplotly()

In [ ]:
results = d_model.run(solver=NumPySSASolver)
results.plotplotly()

# Testing Issue 606 Fix

In [ ]:
d_model.timespan(np.arange(0, 101))

In [ ]:
solvers = [NumPySSASolver, ODESolver, TauLeapingSolver, TauHybridSolver,
    SSACSolver, ODECSolver, TauLeapingCSolver, TauHybridCSolver]

for solver in solvers:
    sol = solver()
    kwargs = {'timeout': 0} if solver is TauHybridSolver else {}
    results = sol.run(model=d_model, **kwargs, t=100, increment=1)
    results.plot(title=sol.name)

# Testing Issue 605 Fix

In [ ]:
d_model.timespan(np.arange(0, 101))
results = d_model.run(solver=NumPySSASolver)
results.plot()

In [ ]:
solver = NumPySSASolver()
results = solver.run(model=d_model, t=100, increment=1)
results.plot()